In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Load data from CSV (replace with your actual file path)
trigger_data = pd.read_csv(r'CSV_Waveforms/waveform_15.11.2024_10.08.29_channel_1.csv')

# Assuming the first column is time and the second is the signal
time_data = trigger_data.iloc[:, 0].values
signal_data = trigger_data.iloc[:, 1].values

# Set the threshold for identifying a trigger event
threshold = 0.5  # Adjust this threshold based on your signal characteristics

# Step 1: Identify all trigger events based on a threshold
trigger_indices = np.where(signal_data > threshold)[0]

# Step 2: Find the first trigger that follows a 1-second quiet period
valid_trigger_index = None
quiet_period_duration = 1.0  # 1 second

for i in trigger_indices:
    # Define the start and end time for the 1-second interval before this trigger
    trigger_time = time_data[i]
    quiet_start_time = trigger_time - quiet_period_duration
    
    # Find indices in this 1-second interval
    quiet_period_indices = np.where((time_data >= quiet_start_time) & (time_data < trigger_time))[0]
    
    # Check if all values in this interval are below the threshold (i.e., no triggers in the last second)
    if np.all(signal_data[quiet_period_indices] < threshold):
        valid_trigger_index = i
        break

if valid_trigger_index is None:
    raise ValueError("No valid trigger found after a 1-second quiet period.")
else:
    # Set the time of this trigger as t=0
    t0_reference = time_data[valid_trigger_index]
    print("Valid trigger found. Setting t=0 at this trigger event.")

# Adjust the time axis so that t0_reference becomes t=0
adjusted_time_data = time_data - t0_reference

# Intermediate Plot: Show the t=0 mark on the original raw data
plt.figure(figsize=(10, 6))
plt.plot(time_data, signal_data, label="Raw Signal Data")
plt.axvline(x=t0_reference, color='r', linestyle='--', label="t = 0 (First Valid Trigger)")
plt.xlabel("Time (s)")
plt.ylabel("Signal Amplitude")
plt.title("Original Signal Data with t=0 Marked")
plt.legend()
plt.grid()
plt.show()

# Step 3: Find the last trigger point within the same sweep
# We only look at triggers that occur after t=0
post_t0_trigger_indices = trigger_indices[trigger_indices >= valid_trigger_index]

last_valid_trigger_index = None
for i in range(1, len(post_t0_trigger_indices)):
    # Define the start and end time for the 1-second interval after this trigger
    last_trigger_time = adjusted_time_data[post_t0_trigger_indices[i]]
    quiet_end_time = last_trigger_time + quiet_period_duration
    
    # Find indices in this 1-second interval after the trigger
    quiet_period_indices_after = np.where((adjusted_time_data > last_trigger_time) & (adjusted_time_data <= quiet_end_time))[0]
    
    # Check if all values in this interval are below the threshold
    if np.all(signal_data[quiet_period_indices_after] < threshold):
        last_valid_trigger_index = post_t0_trigger_indices[i - 1]  # Last trigger before the quiet period
        break

if last_valid_trigger_index is None:
    raise ValueError("No last trigger found within the same sweep (1-second quiet period after last trigger).")
else:
    last_trigger_time = adjusted_time_data[last_valid_trigger_index]
    print("Last trigger within the same sweep found. Ending sweep at this trigger.")

# Step 4: Filter the data from t=0 to the last valid trigger
valid_time_indices = (adjusted_time_data >= 0) & (adjusted_time_data <= last_trigger_time)
valid_time_data = adjusted_time_data[valid_time_indices]
valid_signal_data = signal_data[valid_time_indices]

# Step 5: Convert time to wavelength
start_wavelength = 1535  # starting wavelength in nm
sweep_speed = 20  # nm/s
wavelength_data = start_wavelength + sweep_speed * valid_time_data

# Final Plot: Triggers vs. Wavelength
plt.figure(figsize=(10, 6))
plt.plot(wavelength_data, valid_signal_data, label="Triggers")
plt.xlabel("Wavelength (nm)")
plt.ylabel("Trigger Amplitude (V)")
plt.title("Triggers vs. Wavelength")
plt.grid()
plt.legend()
plt.show()

optical_data = pd.read_csv(r'CSV_Waveforms/waveform_15.11.2024_10.08.29_channel_2.csv')
optical_time_data = optical_data.iloc[:, 0].values - t0_reference  # Align optical data to t=0
optical_power_data = optical_data.iloc[:, 1].values

# Filter optical power data within the time range from t=0 to the last trigger
valid_optical_indices = (optical_time_data >= 0) & (optical_time_data <= last_trigger_time)
valid_optical_time = optical_time_data[valid_optical_indices]
valid_optical_power = optical_power_data[valid_optical_indices]

# Convert valid optical time data to wavelength
valid_optical_wavelength = start_wavelength + sweep_speed * valid_optical_time

# Final Plot: Optical Power vs. Wavelength
plt.figure(figsize=(10, 6))
plt.plot(valid_optical_wavelength, valid_optical_power, label="Optical Power")
plt.xlabel("Wavelength (nm)")
plt.ylabel("Optical Power (V)")
plt.title("Optical Power vs. Wavelength")
plt.grid()
plt.legend()
plt.show()